In [ ]:
from __future__ import division, print_function
%matplotlib inline
%load_ext Cython

# Denoise reproduction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Types of image noise

In [ ]:
noise_types = ['gaussian',
               'poisson',
               's&p',
               'speckle']

astro = img_as_float(data.astronaut())

# Detail patch
astro0 = astro[300:450, 100:320]

fig, ax = plt.subplots(nrows=5, figsize=(3, 10))

pretty_subplot(ax[0], astro0, 'original')

gauss = random_noise(astro0, mode='gaussian', seed=42,
                     var=0.15)
poisson = random_noise(astro0, mode='poisson', seed=42)
saltpepper = random_noise(astro0, mode='s&p', amount=0.2)
speckle = random_noise(astro0, mode='speckle', var=0.15)

pretty_subplot(ax[1], gauss, 'gaussian')
pretty_subplot(ax[2], poisson, 'poisson')
pretty_subplot(ax[3], saltpepper, 'salt & pepper')
pretty_subplot(ax[4], speckle, 'speckle')

fig.tight_layout()
plt.show()
fig.savefig('./noise_types.png', dpi=300, bbox_inches=0)

In [ ]:
random_noise?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import data, img_as_float
from skimage.restoration import (denoise_tv_chambolle, denoise_bilateral,
                                 denoise_wavelet, estimate_sigma)
from skimage.util import random_noise


# astro = img_as_float(data.astronaut())
# astro = astro[220:300, 220:320]
astro=astro0

sigma = 0.3
noisy = random_noise(astro, 'speckle', var=sigma**2, seed=42)

fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(5, 7.3))

def pretty_subplot(ax, image, title=None, ylabel=None):
    ax.imshow(image)
    ax.set_xticks([]) 
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    if title is not None:
        ax.set_title(title)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

pretty_subplot(ax[0, 0], astro, 'original')
pretty_subplot(ax[0, 1], noisy, 'noisy')

pretty_subplot(ax[1, 0], 
               denoise_tv_chambolle(noisy, weight=0.05, 
                                    multichannel=True),
               'weight: 0.05',
               'TV-Chambolle')
pretty_subplot(ax[1, 1], 
               denoise_tv_chambolle(noisy, weight=0.1, 
                                    multichannel=True),
               'weight: 0.1')

pretty_subplot(ax[2, 0],
               denoise_bilateral(noisy, sigma_color=0.05, sigma_spatial=15),
               r'$\sigma_{color}$:0.05, $\sigma_{spatial}$:25',
               'Bilateral')
pretty_subplot(ax[2, 1],
               denoise_bilateral(noisy, sigma_color=0.1, sigma_spatial=20),
               r'$\sigma_{color}$:0.1, $\sigma_{spatial}$:20')

# Estimate sigma for wavelet
sigma_est = estimate_sigma(noisy, multichannel=True, average_sigmas=True)
pretty_subplot(ax[3, 0],
               denoise_wavelet(noisy, sigma=sigma_est,
                               multichannel=True),
               r'$\sigma$: estimated',
               'Wavelet')
pretty_subplot(ax[3, 1],
               denoise_wavelet(noisy, sigma=1.4*sigma_est,
                               multichannel=True),
               r'$\sigma$: 1.4*estimated')

fig.tight_layout(h_pad=0.7, w_pad=0.3)

plt.show()

fig.savefig('./denoise.png', dpi=300, bbox_inches=0)